<a href="https://colab.research.google.com/github/Loga19818eeanvesh/CNN_Models/blob/main/2018_DenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow --upgrade
!pip install keras --upgrade

     |████████████████████████████████| 462 kB 6.9 MB/s 


In [2]:
import tensorflow as tf 
print(tf.__version__)

2.8.0


In [15]:
from tensorflow.keras import Model
from tensorflow.keras.layers import AveragePooling2D,DepthwiseConv2D,Concatenate,Conv2D,BatchNormalization,GlobalAveragePooling2D,Flatten,Add,InputLayer,ReLU,ZeroPadding2D,MaxPool2D,Dense
from tensorflow.keras.activations import relu,softmax

In [9]:
class BottleNeckBlockFor3by3Conv(Model):
  def __init__(self,k):
    super().__init__()
    self.batch_norm_1by1 = BatchNormalization()
    self.relu_activation = ReLU()
    self.conv_1by1 = Conv2D(filters=(4*k),kernel_size=1)
    self.batch_norm_3by3 = BatchNormalization()
    self.conv_3by3 = Conv2D(filters=k,kernel_size=3,padding='same')

  def call(self,inputs):
    x = self.batch_norm_1by1(inputs)
    x = self.relu_activation(x)
    x = self.conv_1by1(x)
    x = self.batch_norm_3by3(x)
    x = self.relu_activation(x)
    x = self.conv_3by3(x)
    return x

In [22]:
class DenseBlock(Model):

  def __init__(self,k,n):
    super().__init__()
    self.n = n
    self.bottle_neck_conv3by3 = {}
    for i in range(n):
      self.bottle_neck_conv3by3["bottle_neck_conv3by3_"+str(i)] = BottleNeckBlockFor3by3Conv(k)
    self.concat = Concatenate()

  def call(self,inputs):
    outputs = {}
    for i in range(self.n):
      outputs["x"+str(i)] = 0

    for i in range(self.n):
      x = [inputs]
      for j in range(i):
        x.append(outputs["x"+str(j)])
      x = self.concat(x)
      outputs["x"+str(i)] = self.bottle_neck_conv3by3["bottle_neck_conv3by3_"+str(i)](x)

    output = []
    for i in range(self.n):
      output.append(outputs["x"+str(i)])
    output = self.concat(output)
    return output

In [14]:
class TransitionLayer(Model):
  def __init__(self,k):
    super().__init__()
    self.batch_norm_1by1 = BatchNormalization()
    self.conv_1by1 = Conv2D(filters=(4*k),kernel_size=1)
    self.avg_pool = AveragePooling2D(pool_size=(2, 2), strides=2)

  def call(self,inputs):
    x = self.batch_norm_1by1(inputs)
    x = self.conv_1by1(x)
    x = self.avg_pool(x)

    return x

In [23]:
class DenseNet(Model):

  def __init__(self,input_shape=(None,224,224,3),k=32,noof_classes=1000):
    super().__init__()
    self.input1 = InputLayer(input_shape=input_shape)
    self.zero_pad = ZeroPadding2D(padding=((3, 2), (3, 2)))
    self.batch_norm_7by7 = BatchNormalization()
    self.relu_activation = ReLU()
    self.conv_7by7 = Conv2D(filters=(2*k),kernel_size=7,padding='valid',strides=2)
    self.batch_norm_pool = BatchNormalization()
    self.maxpool = MaxPool2D(pool_size=3,strides=2)
    self.dense_block1 = DenseBlock(k=k,n=6)
    self.transition_layer1 = TransitionLayer(k=k)
    self.dense_block2 = DenseBlock(k=k,n=12)
    self.transition_layer2 = TransitionLayer(k=k)
    self.dense_block3 = DenseBlock(k=k,n=24)
    self.transition_layer3 = TransitionLayer(k=k)
    self.dense_block4 = DenseBlock(k=k,n=16)
    self.global_avg_pool = GlobalAveragePooling2D()
    self.flatten = Flatten()
    self.classifier = Dense(units=noof_classes,activation=softmax)

  def call(self,inputs):
    x = self.input1(inputs)
    x = self.zero_pad(x)
    x = self.batch_norm_7by7(x)
    x = self.relu_activation(x)
    x = self.conv_7by7(x)
    x = self.batch_norm_pool(x)
    x = self.relu_activation(x)
    x = self.maxpool(x)
    x = self.dense_block1(x)
    x = self.transition_layer1(x)
    x = self.dense_block2(x)
    x = self.transition_layer2(x)
    x = self.dense_block3(x)
    x = self.transition_layer3(x)
    x = self.dense_block4(x)
    x = self.global_avg_pool(x)
    x = self.flatten(x)
    x = self.classifier(x)
    return x


In [24]:
DenseNet_model = DenseNet()

In [25]:
DenseNet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                       loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                       metrics=tf.keras.metrics.SparseCategoricalAccuracy()
                        )

In [26]:
DenseNet_model.build(input_shape=(None,224,224,3))

In [27]:
DenseNet_model.summary()

Model: "dense_net_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, 224, 224,   0         
                             3)]                                 
                                                                 
 zero_padding2d_1 (ZeroPaddi  multiple                 0         
 ng2D)                                                           
                                                                 
 batch_normalization_121 (Ba  multiple                 12        
 tchNormalization)                                               
                                                                 
 re_lu_59 (ReLU)             multiple                  0         
                                                                 
 conv2d_120 (Conv2D)         multiple                  9472      
                                                       